In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_simple_coo_2020.npz").tocsr()

In [2]:
%load_ext Cython

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

np.random.seed(41148)

In [4]:
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_AsySVD_Cython

In [5]:
recommender= MatrixFactorization_AsySVD_Cython(URM_all)

MatrixFactorization_AsySVD_Cython_Recommender: URM Detected 1079 (4.15 %) cold items.


In [6]:
recommender.fit(sgd_mode='adam', epochs=200, use_bias=False, batch_size=1, num_factors=152, item_reg=0.007731021254254606, user_reg=0.00254626255849159, learning_rate= 0.0029212052957107992, negative_interactions_quota=0.06462267789717602)

ASY_SVD: Estimating user factors... 
ASY_SVD: Estimating user factors... done!
ASY_SVD: Processed 113268 ( 100.00% ) in 15.98 seconds. MSE loss 1.03E-01. Sample per second: 7086
ASY_SVD: Epoch 1 of 200. Elapsed time 15.61 sec
ASY_SVD: Processed 113268 ( 100.00% ) in 17.22 seconds. MSE loss 1.14E-01. Sample per second: 6578
ASY_SVD: Epoch 2 of 200. Elapsed time 31.85 sec
ASY_SVD: Processed 113268 ( 100.00% ) in 16.17 seconds. MSE loss 1.10E-01. Sample per second: 7007
ASY_SVD: Epoch 3 of 200. Elapsed time 47.79 sec
ASY_SVD: Processed 113268 ( 100.00% ) in 16.49 seconds. MSE loss 1.06E-01. Sample per second: 6868
ASY_SVD: Epoch 4 of 200. Elapsed time 1.07 min
ASY_SVD: Processed 113268 ( 100.00% ) in 15.95 seconds. MSE loss 1.01E-01. Sample per second: 7103
ASY_SVD: Epoch 5 of 200. Elapsed time 1.33 min
ASY_SVD: Processed 113268 ( 100.00% ) in 16.99 seconds. MSE loss 9.45E-02. Sample per second: 6666
ASY_SVD: Epoch 6 of 200. Elapsed time 1.59 min
ASY_SVD: Processed 113268 ( 100.00% ) in 1

In [7]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)

for uid in target_user_list:
    rec_list=map(str,recommender.recommend(uid)[:10])
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(rec_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("MF_AsySVD_recommendations.csv")